# <p style="text-align: center;"> <span style="color:yellowgreen"> MM orientation </span></p>

At the end of lab 2, you have obtain the line segments representing the white lanes in Duckietown  
In this notebook, we will obtain the orientation values $\varphi$ of the MM based on these line segments, i.e., it measures the MM angle relative to the lane  
We will cover two ways: one using the Duckiebot code, and one using yours  
Feel free to use either of them to complete lab 3 tasks  

**REMARK** This is a real robot with real hardware, so there will be errors in sensing.  
Sensing erros tend to get worse when controlling the robot   
However, there are techniques to mitigate them   
Be patient and work out what your robot is telling you versus what it should be doing  
Here it is a good time to just observe your sensor values and get a feel of their errors  
For instance, start rotating the MM in the lane and see the orientation values changing   
Do you spot any errors? How could we reduce them?


## <p style="text-align: center;"> <span style="color:coral"> Orientation with Duckiebot code </span></p>

In the lab3 package, we have provide a launch file **lab3.launch**  
As it is provide, this launch file starts the lane detector and lane filter (calculates the orientation $\varphi$ of the robot) using the duckiebot code  
The code below is the **lab3.launch** file

In [ ]:
<launch>

    <arg name="veh" default="$(env VEHICLE_NAME)"/>
    <arg name="ai_trafo_mode" default="cb" doc="'cb' for colo balance only; 'both' for color balance and linear trafo"/>
    <arg name="ai_interval" default="5" doc="interval with which the linear trafo gets updated. color balance is performed every second."/>
    <arg name="verbose" default="false"/>

    <!-- start Duckietown nodes -->
    <arg name="demo_name" value="lane_following"/>

    <include file="$(find led_emitter)/launch/led_emitter_node.launch">

        <arg name="veh" value="$(env VEHICLE_NAME)"/>

    </include>
    
    <!-- start basic args -->
    <include file="$(find duckietown_demos)/launch/master.launch">

        <!-- Basic arguments -->
        <arg name="veh" value="$(arg veh)"/>
        <arg name="demo_name" value="$(arg demo_name)"/>
        <arg name="param_file_name" value="default" />
        <arg name="visualization" value="true" />

        <!-- Finite state machine -->
        <arg name="fsm" value="true"/>
        <arg name="/fsm/logic_gate" value="false"/>

        <!-- Camera and anti intagram -->
        <arg name="/camera/raw" value="false" />
        <arg name="anti_instagram" value="true" />

        <!-- Lane Following stack -->
        <arg name="lane_following" value="true"/>
		<!-- The duckiebot lane detector. Initially as true (Active). If you want to use your lane detector, change the value to "false" and start your node in the next group in line 52-->
        <arg name="/lane_following/line_detection" value="true"/>
        
		<arg name="line_detector_param_file_name" value="default" />
        <arg name="/lane_following/ground_projection" value="true"/>

		<!-- Gets line segments from line_detection and transforms to orientation value phi -->
        <arg name="/lane_following/lane_filter" value="true"/>

		<!-- Starts the duckiebot PID - start as false. If you want to check the duckiebot PID change this to true. Remember to change to false when testing your controller -->
        <arg name="/lane_following/lane_controller" value="false"/>

    </include>

    <group ns="$(env VEHICLE_NAME)">
        <!-- UNCOMMENT The line below to start  your lane detection node instead of duckiebots. Update the parameters as needed and make sure to change the /lane_following/line_detection value to false in line 37-->
		<!-- <node pkg = "lab2" type = "line_detection.py" name = "line_detection" output = "screen"/> -->

		<!-- Add your controller and odometry nodes below -->
        

    </group>

</launch>

- Run this launch file with the MM in the track (At the end of this notebook we recall the commands to test code in the MM)

- Start the the gui in a new terminal `dts start_gui_tools ee483mm[number]`
	- Observe the output of topic `rostopic echo ee483mm<number>/lane_filter_node/lane_pose` 
	- It has multiple parameters, but we are interested in the orientation `phi` values  

<br><img src="../assets/lab3-images/echo_lane_pose.png" style="width: 75%; height: auto;"></br>

- Ploting the orientation values 
	- Close the rostopic echo but leave the start_gui_tools terminal open
	- Run the command `rosrun rqt_plot rqt_plot`
	- A window will open: at the top write: `ee483mm<number>/lane_filter_node/lane_pose/phi` 
	- This will plot the values of `phi` through time

<br><img src="../assets/lab3-images/rqt_plot_phi.png" style="width: 75%; height: auto;"></br>

- Rotate the MM in the track to observe the `phi` values

Take note of some observations based on the readings:
- Does it have errors? e.g., offset, sporadic jumps
- When do usually these errors occur? Rotating left, rotating right, etc. 
- What would be possible solutions to get more consistent results?

**REPORT: Add this information to your lab report as well as submit these findings in the Class teams group chat**



## <p style="text-align: center;"> <span style="color:coral"> Orientation with your code </span></p>

You have developed a code to obtain the line segments of white lanes  
To calculate the orientation of the MM using your lane detector we need to add new functions on top of your calculations:
1. Detect the yellow lanes (for better accuracy)
2. Publish normalized white and yellow line segments in a topic

We have created a node called *line_detection.py* inside the lab 3 package  
You can your color filtering, canny, and HoughLinesP tunning values from lab 2 in
 - Function white_filter   
	- Lines 60 - Your coloring filter HSV values
	- Line 61 - Your erosion kernel
	- Line 62 - Your dilation kernel
 - Function segment  
	- Lines 93 - Your Canny thresholds
	- Line 108 - Your HoughLinesP values

You can also tune the yellow filter if needed.  
You do not need to change the cropping (This is fixed)  
The image has also been downgraded for better performance  


Once you have updated your values:
- Change lines 36 (value = "false") and 51 (uncomment) 
- Run your the launch file (after dts devel run -H mmname.local -s -f -M --cmd bash)
- Start GUI in a new terminal "dts start_gui_tools mmname"   
- Run either rostopic echo or rqt_plot (as described in the first method above) to see phi values 



**REPORT:Which one of codes (yours or duckiebot's) provide better results?**


## Reminders to test code in the MM

### Testing code

Similar to Lab 2 after uploading your Docker image to MM, we need to run the following command to update any changes you make in your lab3 package. 

**Assumptions:**
- There is the lab3 package
- You are running the command inside `lab-devel`
- The laptop is connected the `robot-lan` network or wired to the MM
- The MM is on


In [ ]:
dts devel run -H ee483mm[number].local -s -M --cmd bash -f 

Once the MM shell appear, built and activate the catkin workspace  
Sometimes, you don't need to build or source it everytime you run dts devel run (give a try)  
It might be that the autocomplete might not work though but the rosrun still works with your package when you run it

In [ ]:
catkin_build lab3
source /code/catkin_ws/devel/setup.bash

### Common errors
1. If you get a docker conflict saying container is already running, there are two options 

	- Attach to the running container.

		`dts devel run attach`

	- Delete the running container. Do this if you have updates 

		- Connect to shell: ssh duckie@\<MMname\>.local 

		- List docker images: docker ps 

		- Delete the container: docker rm -f \<container_ID\> 

		- Exit the shell: exit 


### GUI TOOLS
In a hostOS terminal run the following commands

In [ ]:
dts start_gui_tools ee483mm[number]

1. To observe images: `rqt_image_view`
2. To observe the rqt_plot: `rqt_plot`
3. To plot topic values: `rosrun rqt_plot rqt_plot`
4. To observe topic values in the terminal: `rostopic echo [name of the topic]`


<br><img src="../assets/lab3-images/dts-gui.png" style="width: 75%; height: auto;"></br>